In [343]:
# Import necessary modules
#pip install langdetect
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import langdetect 
import pandas as pd
import nltk
from langdetect import detect

final_data = pd.read_csv(r"C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\final_data.csv", index_col=0).reset_index()
df = final_data.copy()

## Sample Control
We draw multiple random samples with 10 links each. We check that the jobs we have classified as remote are indeed remote by going in each link and reading through the complete job listing. Notice that many of them are broken links which means we need to go to Jobindex and check whether they kept a copy of the listing.

In [398]:
df['Link'].sample(n=10)

4323    https://dk.indeed.com/viewjob?jk=3c6f929aa17c3...
5172    https://www.jobindex.dk/jobannonce/273347/vice...
3272    http://www.hammerandersen.com/list/372/ledige-job
8049    https://www.regnskabsspecialisten.dk/controlle...
8446           http://go.dk/job/638346/c3-consulting-aps/
4694    http://jobbank.dk/job/664674/university-of-cam...
8027    http://www2.forsvaret.dk/job/ledige_stillinger...
1138    https://careers.peopleclick.com/careerscp/clie...
1591                   https://www.hansentoft.dk/job/3842
4611        https://thehub.dk/jobs/social-media-intern-29
Name: Link, dtype: object

## Text Analysis

We begin by cleaning the snippets of job descriptions as much as possible by removing noisy terms such as symbols, numbers and stopwords. Part of this preprocessing also consists in separating the Danish and English job descriptions such that we can remove the relevant stop words for each language and also find the most common words for each language.

In [261]:
# Removing job title and company info from the description column
# Detecting language
df['Language'] = ''
for i in range(len(df)):
    #df['Description'][i] = str(df['Description'][i]).replace(df['Job Title'][i],'')
    df['Description'][i] = str(df['Description'][i]).replace(df['Company'][i],'')
    df['Description'][i] = str(df['Description'][i]).strip()
    if len(df['Description'][i]) > 10:
        df['Language'][i] = detect(str(df['Description'][i]))
    else:
        df['Language'][i] = 'da'

C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launc

In [262]:
# Some job postings were mistakenly classified as languages other than english or danish when they were actually danish:

for i in range(len(df)):
    if df['Language'][i] != 'da' and df['Language'][i] != 'en': 
        df['Language'][i] = 'da'
        
from nltk.stem.snowball import SnowballStemmer
stemmer_da = SnowballStemmer("danish")
stemmer_en = SnowballStemmer("english")

C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [331]:
# Now we clean the descriptions as much as we can, tokenize them and stem them using the snowball stemmer
symbols = ['?','!','>','<','-','[',']','(',')','{','}',' –','``',"''",'""','\\','@','$','&','=']
useless_words = ['søger', 'vores', '.', 'kunder','jobcenter','arbejde','dine', 'får', 'samt', 'del','inden', 's', 'danmark','dit', 'både', 'ved','inden' ,'søges', 'a', 's', 'få', 'kan', 'så', 'opgaver',\
                 'jobsincopenhagen' ,'looking', 'denmark', 'work','company','eksiterende','spændende']
df['Tokenized_description'] = ''
for i in range(len(df)):
    stop_words_da = nltk.corpus.stopwords.words("danish")
    stop_words_en = nltk.corpus.stopwords.words("english")
    df['Tokenized_description'][i] = re.sub(r'(\.+ )|,|\||:|/|\'|\-|;|\*|!|(\s\d+\s)|(\s\W\s)',' ',str(df['Description'][i]))
    df['Tokenized_description'][i] = str(df['Tokenized_description'][i]).rstrip('\\')
    df['Tokenized_description'][i] = nltk.word_tokenize(str(df['Tokenized_description'][i].lower()))
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in symbols]
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w.isnumeric()]
    df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in useless_words]
    if df['Language'][i] == 'da':
        df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in stop_words_da]
        #df['Tokenized_description'][i] = [stemmer_da.stem(w) for w in df['Tokenized_description'][i]]
    else:
        df['Tokenized_description'][i] = [w for w in df['Tokenized_description'][i] if not w in stop_words_en]
        #df['Tokenized_description'][i] = [stemmer_en.stem(w) for w in df['Tokenized_description'][i]]

C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lisbe\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added 

In [332]:
words_da_tok = df_da['Tokenized_description'].tolist()
words_en_tok = df_en['Tokenized_description'].tolist()

# Putting all words from all documents into the same list
flat_list_da = [item for sublist in words_da_tok for item in sublist] 
flat_list_en = [item for sublist in words_en_tok for item in sublist]

#Finding the most common words
from collections import Counter
top_en = Counter(flat_list_en).most_common(20)
top_da = Counter(flat_list_da).most_common(20)

In [333]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Since we have already pre processed our data, we will use the vectorizer without it tokenizing our data
def dummy_fun(doc):
    return doc
# We split the datasets according to language
df_da = df[df['Language']=='da']
df_en = df[df['Language']=='en']

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    max_df=0.95,
    min_df=0.04,
    max_features=10)  

vectors_da = tfidf.fit_transform(words_da_tok)
vocabulary_da = tfidf.vocabulary_
print(vocabulary_da)

vectors_den = tfidf.fit_transform(words_en_tok)
vocabulary_en = tfidf.vocabulary_
print(vocabulary_en)

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
bag = count.fit_transform(df['Tokenized_description'])
print(count.vocabulary_)